### RAG Citation: Enhancing AI-Generated Content with Automatic Citations (A Non-LLM Approach)
#### 2. Example-embeddchain

In [7]:
## install embedchain
# !pip install embedchain

In [3]:
## install rag_citation
# !pip install rag_citation

#### 1. add your OpenAI key

* create .env file
* add your OPENAI_API_KEY='sk-****'

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### 2. ingesting documents

In [3]:
from embedchain import App

app = App()
app.add("https://www.forbes.com/profile/elon-musk")
app.add("https://en.wikipedia.org/wiki/Elon_Musk")

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2024-11-04 23:08:04,803 - 8421618240 - local_persistent_hnsw.py-local_persistent_hnsw:271 - WARNING: Add of existing embedding ID: default-app-id--4f7d3091513cc3bc304ce8eb6b98272c8e0b0d4d42d5f9d61b9faba9b5295545
2024-11-04 23:08:04,804 - 8421618240 - local_persistent_hnsw.py-local_persistent_hnsw:271 - WARNING: Add of existing embedding ID: default-app-id--bc7ce15b75da8e1bb3f0b24f3e1126afc545e2934893b611001b68c539b84f4e
2024-11-04 23:08:04,805 - 8421618240 - local_persistent_hnsw.py-local_persistent_hnsw:271 - WARNING: Add of existing embedding ID: default-app-id--a9ed7a850be9ba86a7aaebf3926ca6985188bd2fb79cd0c4123b992104fe952f
Inserting batches in chromadb: 100%|██████████| 2/2 [00:07<00:00,  3.56s/it]


'1eab8dd1ffa92906f7fc839862871ca5'

#### 3. Query

In [4]:
app.query("What is the net worth of Elon Musk today?")

2024-11-04 23:08:18,110 - 8421618240 - embedchain.py-embedchain:558 - WARNING: Starting from v0.1.125 the return type of query method will be changed to tuple containing `answer`.


"Elon Musk's net worth is estimated to be approximately $260.9 billion as of November 4, 2024."

In [5]:
documents=app.search("What is the net worth of Elon Musk today?")

#### 4. rag-citation

In [6]:
documents

[{'context': "Elon Musk PROFILEElon MuskCEO, Tesla$260.9B$2.4B (0.91%)Real Time Net Worthas of 11/4/24Reflects change since 5 pm ET of prior trading day. 1 in the world todayPhoto by Martin Schoeller for ForbesFrom the EditorElon Musk cofounded seven companies, including electric car maker Tesla, rocket producer SpaceX and artificial intelligence startup xAI.He owns about 12% of Tesla excluding options, but has pledged more than half his shares as collateral for personal loans of up to $3.5 billion.In early 2024, a Delaware judge voided Musk's 2018 deal to receive options equaling an additional 9% of Tesla. Forbes has discounted the options by 50% pending Musk's appeal.SpaceX, founded in 2002, is worth nearly $210 billion based on a tender offer launched during the second half of 2024. Musk owns an estimated 42% stake.Musk bought Twitter in a $44 billion (enterprise value) deal during 2022. Forbes estimates that the social media company, which he renamed X, is worth nearly 70% less as 

In [7]:
document=[]
for x in documents:
    document.append({
         "source_id": x["metadata"]["doc_id"],
        "document": x["context"],
        "meta": [{"url": x["metadata"]["url"],"chunk_id":x["metadata"]["hash"]}]
    })

In [8]:
document

[{'source_id': 'default-app-id--ff8ca63140946d2f2118daecc6d54afd90a86dacccd5c91d60a3b8689dd2f46a',
  'document': "Elon Musk PROFILEElon MuskCEO, Tesla$260.9B$2.4B (0.91%)Real Time Net Worthas of 11/4/24Reflects change since 5 pm ET of prior trading day. 1 in the world todayPhoto by Martin Schoeller for ForbesFrom the EditorElon Musk cofounded seven companies, including electric car maker Tesla, rocket producer SpaceX and artificial intelligence startup xAI.He owns about 12% of Tesla excluding options, but has pledged more than half his shares as collateral for personal loans of up to $3.5 billion.In early 2024, a Delaware judge voided Musk's 2018 deal to receive options equaling an additional 9% of Tesla. Forbes has discounted the options by 50% pending Musk's appeal.SpaceX, founded in 2002, is worth nearly $210 billion based on a tender offer launched during the second half of 2024. Musk owns an estimated 42% stake.Musk bought Twitter in a $44 billion (enterprise value) deal during 20

In [9]:
answer="Elon Musk's net worth is estimated to be US$241 billion as of August 2024."


In [10]:
from rag_citation import CiteItem, Inference

In [11]:
inference = Inference(spacy_model="sm", embedding_model="md")
cite_item = CiteItem(answer=answer, context=document)

In [12]:
output=inference(cite_item)

100%|██████████| 2/2 [00:00<00:00,  2.27it/s]


In [13]:
output.citation

[{'answer_sentences': "Elon Musk's net worth is estimated to be US$241 billion as of August 2024.",
  'cite_document': [{'document': 'He is one of the wealthiest individuals in the world; as of August 2024[update]',
    'source_id': 'default-app-id--c52afee8ceb3c20a3099b52a3a6a3cacf561f941183d4bfb62c3905b80a00a2d',
    'entity': [{'word': 'August 2024', 'entity_name': 'DATE'}],
    'score': 100,
    'meta': [{'url': 'https://en.wikipedia.org/wiki/Elon_Musk',
      'chunk_id': '1eab8dd1ffa92906f7fc839862871ca5'}]}]}]

In [14]:
output.missing_word

['US$241 billion']

In [15]:
output.hallucination

True